In [1]:
# import all the libraries

# Data exploration and analysis tools
import pandas as pd
import seaborn as sns
import numpy as np
from ast import literal_eval
import re as re

# Data cleaning for machine learning models
from sklearn.model_selection import train_test_split #split data into testing and training data
from sklearn.feature_selection import SelectKBest # identify best X that may predict Y
from sklearn.feature_selection import mutual_info_regression #needed for SelectKBest
from sklearn.preprocessing import StandardScaler #handle outliers after selecting K best guess variables that predict Y

# Machine Learning model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
#reg = LinearRegression(fit_intercept=True)
#fit_intercept = True; hyper parameter for linear regression, add one-extra term - a start value (a starting weight); rarely False



# Error Measures
from sklearn.dummy import DummyRegressor
# Use DummyRegressor to compare your linear regression to the dumbest possible

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error



In [2]:
bike_data = pd.read_csv('bike_index_api_stolenessall.csv')

In [3]:
bike_df = pd.DataFrame(data=bike_data)

In [4]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188574 entries, 0 to 188573
Data columns (total 14 columns):
Unnamed: 0           188574 non-null int64
date_stolen          62097 non-null float64
frame_colors         188574 non-null object
frame_model          170000 non-null object
id                   188574 non-null int64
is_stock_img         188574 non-null bool
large_img            78875 non-null object
manufacturer_name    188567 non-null object
serial               188211 non-null object
stolen               188574 non-null bool
stolen_location      61129 non-null object
thumb                78875 non-null object
title                188573 non-null object
year                 133836 non-null float64
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 17.6+ MB


This bike data has all datapoints ever collected in the BikeIndex API. What I really want are datapoints for bikes that are in San Francisco. For the purposes of this project, I am going to narrow this data set to bikes that have some representation of California. Of the data that I know is in California, there will be some NaNs as I'm digging into San Francisco data specifically.

In [5]:
bike_df_clean = bike_df.copy()
bike_df_clean = bike_df_clean.dropna(subset=['stolen_location'])
bike_df_clean = bike_df_clean[bike_df_clean['stolen_location'].apply(str).str.contains('CA|California|San Francisco')]

In [6]:
def split_zipcode(x):
    array=re.findall('\d{5}',str(x))
    if len(array)==0:
        return None
    else:
        return array[0]
            
bike_df_clean['stolen_zipcode'] = bike_df_clean['stolen_location'].apply(split_zipcode)

In [7]:
bike_df_clean

Unnamed: 0   date_stolen                           frame_colors  \
33             33  1.541264e+09                               ['Blue']   
39             39  1.541236e+09  ['Red', 'Silver, gray or bare metal']   
40             40  1.541236e+09         ['Silver, gray or bare metal']   
41             41  1.541221e+09                              ['Black']   
42             42  1.541259e+09                              ['White']   
47             47  1.541200e+09         ['Silver, gray or bare metal']   
48             48  1.541192e+09         ['Silver, gray or bare metal']   
49             49  1.541192e+09         ['Silver, gray or bare metal']   
56             56  1.541185e+09                       ['Blue', 'Blue']   
69             69  1.541131e+09                              ['Black']   
71             71  1.541131e+09         ['Silver, gray or bare metal']   
72             72  1.541131e+09                              ['Black']   
73             73  1.541130e+09                              ['Black']   
78             78  1.541097e+09                               ['Teal']   
93             93  1.541027e+09         ['Silver, gray or bare metal']   
97             97  1.541012e+09                              ['Black']   
98             98  1.541011e+09                       ['Black', 'Red']   
133           133  1.541264e+09                               ['Blue']   
139           139  1.541236e+09  ['Red', 'Silver, gray or bare metal']   
140           140  1.541236e+09         ['Silver, gray or bare metal']   
141           141  1.541221e+09                              ['Black']   
142           142  1.541259e+09                              ['White']   
147           147  1.541200e+09         ['Silver, gray or bare metal']   
148           148  1.541192e+09         ['Silver, gray or bare metal']   
150           150  1.541192e+09         ['Silver, gray or bare metal']   
156           156  1.541185e+09                       ['Blue', 'Blue']   
169           169  1.541131e+09                              ['Black']   
170           170  1.541131e+09                              ['Black']   
171           171  1.541131e+09         ['Silver, gray or bare metal']   
173           173  1.541130e+09                              ['Black']   
...           ...           ...                                    ...   
62883       62883  9.677880e+08                              ['White']   
62891       62891  9.651096e+08                              ['Brown']   
62892       62892  9.636408e+08                              ['White']   
62896       62896  9.599256e+08                               ['Blue']   
62902       62902  9.581112e+08                     ['Black', 'Brown']   
62904       62904  9.571608e+08                      ['Blue', 'White']   
62913       62913  9.537048e+08                      ['Blue', 'Black']   
62914       62914  9.536184e+08                               ['Blue']   
62915       62915  9.535320e+08             ['Black', 'White', 'Blue']   
62920       62920  9.508536e+08                     ['Yellow or Gold']   
62921       62921  9.505944e+08    ['Yellow or Gold', 'Pink', 'Green']   
62923       62923  9.505080e+08         ['Silver, gray or bare metal']   
62925       62925  9.503352e+08                              ['Brown']   
62928       62928  9.493848e+08                     ['Pink', 'Purple']   
62930       62930  9.493848e+08                                ['Red']   
62933       62933  9.483480e+08                    ['Purple', 'Black']   
62938       62938  9.470520e+08                              ['Black']   
62941       62941  9.467064e+08                               ['Blue']   
62942       62942  9.467064e+08                              ['Black']   
62945       62945  9.467064e+08                              ['Black']   
62956       62956  9.149976e+08                                ['Red']   
62958       62958  9.124920e+08                                ['Red']   
62959      

In [8]:
bike_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13313 entries, 33 to 99992
Data columns (total 15 columns):
Unnamed: 0           13313 non-null int64
date_stolen          13313 non-null float64
frame_colors         13313 non-null object
frame_model          12308 non-null object
id                   13313 non-null int64
is_stock_img         13313 non-null bool
large_img            8185 non-null object
manufacturer_name    13313 non-null object
serial               13282 non-null object
stolen               13313 non-null bool
stolen_location      13313 non-null object
thumb                8185 non-null object
title                13313 non-null object
year                 11010 non-null float64
stolen_zipcode       12881 non-null object
dtypes: bool(2), float64(2), int64(2), object(9)
memory usage: 1.4+ MB


In [9]:
bike_df_clean['date_stolen'] = pd.to_datetime(bike_df_clean['date_stolen'],unit='s')

In [10]:
bike_df_clean['year_stolen'] = bike_df_clean['date_stolen'].dt.year

In [11]:
bike_df_clean

Unnamed: 0         date_stolen                           frame_colors  \
33             33 2018-11-03 17:00:00                               ['Blue']   
39             39 2018-11-03 09:00:00  ['Red', 'Silver, gray or bare metal']   
40             40 2018-11-03 09:00:00         ['Silver, gray or bare metal']   
41             41 2018-11-03 05:00:00                              ['Black']   
42             42 2018-11-03 15:24:15                              ['White']   
47             47 2018-11-02 23:00:00         ['Silver, gray or bare metal']   
48             48 2018-11-02 21:00:44         ['Silver, gray or bare metal']   
49             49 2018-11-02 21:00:00         ['Silver, gray or bare metal']   
56             56 2018-11-02 18:56:45                       ['Blue', 'Blue']   
69             69 2018-11-02 04:00:00                              ['Black']   
71             71 2018-11-02 04:00:00         ['Silver, gray or bare metal']   
72             72 2018-11-02 04:00:00                              ['Black']   
73             73 2018-11-02 03:40:19                              ['Black']   
78             78 2018-11-01 18:29:28                               ['Teal']   
93             93 2018-10-31 23:00:00         ['Silver, gray or bare metal']   
97             97 2018-10-31 19:00:00                              ['Black']   
98             98 2018-10-31 18:32:40                       ['Black', 'Red']   
133           133 2018-11-03 17:00:00                               ['Blue']   
139           139 2018-11-03 09:00:00  ['Red', 'Silver, gray or bare metal']   
140           140 2018-11-03 09:00:00         ['Silver, gray or bare metal']   
141           141 2018-11-03 05:00:00                              ['Black']   
142           142 2018-11-03 15:24:15                              ['White']   
147           147 2018-11-02 23:00:00         ['Silver, gray or bare metal']   
148           148 2018-11-02 21:00:44         ['Silver, gray or bare metal']   
150           150 2018-11-02 21:00:00         ['Silver, gray or bare metal']   
156           156 2018-11-02 18:56:45                       ['Blue', 'Blue']   
169           169 2018-11-02 04:00:00                              ['Black']   
170           170 2018-11-02 04:00:00                              ['Black']   
171           171 2018-11-02 04:00:00         ['Silver, gray or bare metal']   
173           173 2018-11-02 03:40:19                              ['Black']   
...           ...                 ...                                    ...   
62883       62883 2000-09-01 06:00:00                              ['White']   
62891       62891 2000-08-01 06:00:00                              ['Brown']   
62892       62892 2000-07-15 06:00:00                              ['White']   
62896       62896 2000-06-02 06:00:00                               ['Blue']   
62902       62902 2000-05-12 06:00:00                     ['Black', 'Brown']   
62904       62904 2000-05-01 06:00:00                      ['Blue', 'White']   
62913       62913 2000-03-22 06:00:00                      ['Blue', 'Black']   
62914       62914 2000-03-21 06:00:00                               ['Blue']   
62915       62915 2000-03-20 06:00:00             ['Black', 'White', 'Blue']   
62920       62920 2000-02-18 06:00:00                     ['Yellow or Gold']   
62921       62921 2000-02-15 06:00:00    ['Yellow or Gold', 'Pink', 'Green']   
62923       62923 2000-02-14 06:00:00         ['Silver, gray or bare metal']   
62925       62925 2000-02-12 06:00:00                              ['Brown']   
62928       62928 2000-02-01 06:00:00                     ['Pink', 'Purple']   
62930       62930 2000-02-01 06:00:00                                ['Red']   
62933       62933 2000-01-20 06:00:00                    ['Purple', 'Black']   
62938       62938 2000-01-05 06:00:00                              ['Black']   
62941       62941 2000-01-01 06:00:00                               ['Blue']   
62942  

I want to know how many years are represented in this data set. My zipcode data set recommends only using information from 2015 onwards.

In [12]:
# Check to see if the data is clean for year, as in, there are no repeat years or strange ways to notate the year
bike_df_clean['year_stolen'].unique()

array([2018, 2009, 2008, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010,
       2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1998, 1990])

In [13]:
year_stolen = []
year_stolen = bike_df_clean['year_stolen'].unique()
year_stolen.sort()
year_stolen

array([1990, 1998, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [14]:
bike_df_clean.groupby('year_stolen').count()

Unnamed: 0  date_stolen  frame_colors  frame_model    id  \
year_stolen                                                             
1990                  1            1             1            1     1   
1998                  6            6             6            6     6   
2000                 20           20            20           17    20   
2001                  2            2             2            1     2   
2002                  6            6             6            6     6   
2003                  2            2             2            2     2   
2004                 10           10            10           10    10   
2005                108          108           108          103   108   
2006                149          149           149          145   149   
2007                199          199           199          189   199   
2008                357          357           357          343   357   
2009                408          408           408          386   408   
2010                493          493           493          456   493   
2011                647          647           647          615   647   
2012                925          925           925          870   925   
2013               1317         1317          1317         1250  1317   
2014               1361         1361          1361         1264  1361   
2015               1365         1365          1365         1278  1365   
2016               1976         1976          1976         1771  1976   
2017               2244         2244          2244         2036  2244   
2018               1717         1717          1717         1559  1717   

             is_stock_img  large_img  manufacturer_name  serial  stolen  \
year_stolen                                                               
1990                    1          0                  1       1       1   
1998                    6          0                  6       6       6   
2000                   20          3                 20      20      20   
2001                    2          1                  2       2       2   
2002                    6          1                  6       6       6   
2003                    2          0                  2       2       2   
2004                   10          0                 10      10      10   
2005                  108         15                108     108     108   
2006                  149         14                149     149     149   
2007                  199         64                199     199     199   
2008                  357        154                357     357     357   
2009                  408        163                408     408     408   
2010                  493        230                493     493     493   
2011                  647        328                647     647     647   
2012                  925        500                925     925     925   
2013                 1317        744               1317    1317    1317   
2014                 1361        975               1361    1361    1361   
2015                 1365       1090               1365    1359    1365   
2016                 1976       1353               1976    1967    1976   
2017                 2244       1430               2244    2234    2244   
2018                 1717       1120               1717    1711    1717   

             stolen_location  thumb  title  year  stolen_zipcode  
year_stolen                                                       
1990                       1      0      1     1               0  
1998                       6      0      6     6               6  
2000                      20      3     20    18              20  
2001                       2      1      2     1               2  
2002                       6      1      6     5               6  
2003                       2      0      2     2               2  
2004                      10      0     10     8              10  
2005

In [15]:
bike_df_clean

Unnamed: 0         date_stolen                           frame_colors  \
33             33 2018-11-03 17:00:00                               ['Blue']   
39             39 2018-11-03 09:00:00  ['Red', 'Silver, gray or bare metal']   
40             40 2018-11-03 09:00:00         ['Silver, gray or bare metal']   
41             41 2018-11-03 05:00:00                              ['Black']   
42             42 2018-11-03 15:24:15                              ['White']   
47             47 2018-11-02 23:00:00         ['Silver, gray or bare metal']   
48             48 2018-11-02 21:00:44         ['Silver, gray or bare metal']   
49             49 2018-11-02 21:00:00         ['Silver, gray or bare metal']   
56             56 2018-11-02 18:56:45                       ['Blue', 'Blue']   
69             69 2018-11-02 04:00:00                              ['Black']   
71             71 2018-11-02 04:00:00         ['Silver, gray or bare metal']   
72             72 2018-11-02 04:00:00                              ['Black']   
73             73 2018-11-02 03:40:19                              ['Black']   
78             78 2018-11-01 18:29:28                               ['Teal']   
93             93 2018-10-31 23:00:00         ['Silver, gray or bare metal']   
97             97 2018-10-31 19:00:00                              ['Black']   
98             98 2018-10-31 18:32:40                       ['Black', 'Red']   
133           133 2018-11-03 17:00:00                               ['Blue']   
139           139 2018-11-03 09:00:00  ['Red', 'Silver, gray or bare metal']   
140           140 2018-11-03 09:00:00         ['Silver, gray or bare metal']   
141           141 2018-11-03 05:00:00                              ['Black']   
142           142 2018-11-03 15:24:15                              ['White']   
147           147 2018-11-02 23:00:00         ['Silver, gray or bare metal']   
148           148 2018-11-02 21:00:44         ['Silver, gray or bare metal']   
150           150 2018-11-02 21:00:00         ['Silver, gray or bare metal']   
156           156 2018-11-02 18:56:45                       ['Blue', 'Blue']   
169           169 2018-11-02 04:00:00                              ['Black']   
170           170 2018-11-02 04:00:00                              ['Black']   
171           171 2018-11-02 04:00:00         ['Silver, gray or bare metal']   
173           173 2018-11-02 03:40:19                              ['Black']   
...           ...                 ...                                    ...   
62883       62883 2000-09-01 06:00:00                              ['White']   
62891       62891 2000-08-01 06:00:00                              ['Brown']   
62892       62892 2000-07-15 06:00:00                              ['White']   
62896       62896 2000-06-02 06:00:00                               ['Blue']   
62902       62902 2000-05-12 06:00:00                     ['Black', 'Brown']   
62904       62904 2000-05-01 06:00:00                      ['Blue', 'White']   
62913       62913 2000-03-22 06:00:00                      ['Blue', 'Black']   
62914       62914 2000-03-21 06:00:00                               ['Blue']   
62915       62915 2000-03-20 06:00:00             ['Black', 'White', 'Blue']   
62920       62920 2000-02-18 06:00:00                     ['Yellow or Gold']   
62921       62921 2000-02-15 06:00:00    ['Yellow or Gold', 'Pink', 'Green']   
62923       62923 2000-02-14 06:00:00         ['Silver, gray or bare metal']   
62925       62925 2000-02-12 06:00:00                              ['Brown']   
62928       62928 2000-02-01 06:00:00                     ['Pink', 'Purple']   
62930       62930 2000-02-01 06:00:00                                ['Red']   
62933       62933 2000-01-20 06:00:00                    ['Purple', 'Black']   
62938       62938 2000-01-05 06:00:00                              ['Black']   
62941       62941 2000-01-01 06:00:00                               ['Blue']   
62942  

In [16]:
bike_df_final = bike_df_clean[bike_df_clean['year_stolen'].isin(['2018','2017','2016','2015'])]

In [17]:
bike_df_final['year_stolen'].unique()

array([2018, 2017, 2016, 2015])